# Fair Housing Group: Week 7 Assignment

**Members: Chase Engelhardt, Christine Bustillos, Victor Tran, Shweta Sundar**

In this assignment we will be pooling our collective cleaned-up work with detailed markdown cells below outputs explaining their significance. Additionally we have created a function that allows us to dissolve census data into neighborhood shapes

In [ ]:
import pandas as pd
import geopandas as gpd
import fiona
import geopandas.geoseries  
import folium
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import contextily as ctx
from shapely import wkt

## This is our function
It takes census tract level geospatial data and dissolves it into neighborhood shapes. 
The arguments you need are your data frame (i.e. 'df= your data frame' and your geometry column, usually named 'geometry' (i.e. "geometry='geometry'")

In [16]:
def nhdissolve(df,geometry):
    
    #Import neighborhood shapes
    url = "https://usc.data.socrata.com/api/geospatial/r8qd-yxsr?method=export&format=GeoJSON"
    nhoods=gpd.read_file(url)
   
    #This is to make the geography column recognizable, if you get an error you can try deleting this line from the code
    df[geometry] = df[geometry].apply(wkt.loads)
    #add a CRS
    gdf = gpd.GeoDataFrame(df, crs="EPSG:4326",geometry='geometry')
     #join race with nh shapes, rename neighborhood column, and set it as the index
    nhjoin=gdf.overlay(nhoods, how='intersection')
    nhjoin=nhjoin.rename(columns={'name':'Neighborhood'})
    nhjoin.set_index('Neighborhood',inplace=True)
    
    #drop shapes not in city of LA
    nhjoin.drop(nhjoin[nhjoin['type'] != 'segment-of-a-city'].index, inplace = True)

    #approximate average for data in neighborhoods 
    nhdata= nhjoin.dissolve(by='Neighborhood', aggfunc='sum')
    nhdata.head()
    return nhdata
    

## Here's an example input:

**test=nhdissolve(df=racedf,geometry='geometry')**

## Once you've run the function with your dataframe:
You can call for your new variable in a map 

In the example above this would be 'test'

I'll run an example below:

In [18]:
# step 1- import your data (needs to have a geometry column) 
testdf=pd.read_csv('data/racetracts.csv')

In [19]:
# Step 2- plug it into the function 
example= nhdissolve(df=testdf,geometry='geometry')

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1676: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = data.groupby(**groupby_kwargs).agg(aggfunc)


In [20]:
# Don't worry about the warning^^ it's just talking about one of the functions that's due for a change
# Step 3- Do whatever you want with your newly adjusted geodataframe 
example.head()

,geometry,Unnamed: 0,CT20,FIPS,TotalPop,Non Hispanic,Non Hispanic White Alone,Non Hispanic Black Alone,Non Hispanic American Indian and Alaska Native Alone,Non Hispanic Asian Alone,Non Hispanic Native Hawaiian and Pacific Islander Alone,Non Hispanic Other Alone,Non Hispanic Multi Race,Hispanic
Neighborhood,,,,,,,,,,,,,,
Adams-Normandie,"POLYGON ((-118.30541 34.02571, -118.30890 34.0...",8267,2439227,66409439227,36861,15352,3804,5859,60,4337,34,245,1013,21509
Arleta,"POLYGON ((-118.43579 34.22987, -118.43582 34.2...",2064,1667245,84519667245,56835,17156,8872,757,91,6513,44,202,677,39679
Arlington Heights,"POLYGON ((-118.33381 34.03501, -118.33271 34.0...",9227,2820162,78483820162,43166,21826,4339,5989,69,9858,21,300,1250,21340
Atwater Village,"POLYGON ((-118.27051 34.12151, -118.27331 34.1...",11440,3729825,72447729825,48540,32363,22006,1051,92,7536,43,184,1451,16177
Baldwin Hills/Crenshaw,"POLYGON ((-118.36504 34.00556, -118.36623 34.0...",24408,7467665,126784467665,73917,51807,9726,33070,117,4061,94,734,4005,22110
